Preparing

In [156]:
#sizes = [0.2, 0.3, 0.3, 0.3, 0.4, 0.5, 0.6]
sizes = [0.5, 0.3, 0.2, 0.4, 0.7, 0.2, 0.4, 0.8, 0.1, 0.1]
#{0.8, 0.2}, {0.7, 0.3}, {0.5, 0.4. 0.1}, {0.4, 0.2, 0.1}
#sizes = [0.5, 0.3, 0.4, 0.2, 0.6]
#sizes = [0.5, 0.4]

class BinPackingEnv:
    def __init__(self, sizes):
        self.sizes = sizes
        self._count = len(sizes)
        self.clearBins()

    def clearBins(self):
        self.bins = [[] for i in range(self._count)]
        self.bin_w = [0] * self._count

    def addItem(self, id : int, bin : int):
        self.bins[bin].append(id)
        self.bin_w[bin] += sizes[id]

    def isOverflow(self):
        for i in range(self._count):
            if self.bin_w[i] > 1:
                return True
        return False
    
    def countValidBin(self):
        count = 0
        for bin in self.bins:
            if len(bin) != 0:
                count += 1
        return count
    def getCount(self):
        return self._count
    
def printPatterns(patterns : list[list[int]]):
    for bins in patterns:
        print('----------')
        for bin in bins:
            print(bin)
            




Solvers

In [157]:
import copy
import itertools
from typing import TypeVar

class BruteForceSolver:
     def solve(self, env:BinPackingEnv):
        patterns = [[(x // pow(env.getCount(), i) % env.getCount()) for i in range(env.getCount())] for x in range(pow(env.getCount(), env.getCount()))]
        min_bin = env.getCount()
        min_patterns = []

        for pattern in patterns:
            overflow = False
            env.clearBins()
            for id in range(env.getCount()):
                env.addItem(id, pattern[id])
                if env.isOverflow():
                    overflow = True
                    break
            if not overflow:
                bin_c = env.countValidBin()
                if min_bin > bin_c:
                    min_patterns.clear()
                    min_bin = bin_c
                    min_patterns.append(pattern)
                    print("least pattern reached : ",pattern)
                elif min_bin == bin_c:
                    min_patterns.append(pattern)

        print(min_patterns)

class BruteForceSolverSingleLoop:
     def solve(self, env:BinPackingEnv):
        #patterns = [[(x // pow(env.getCount(), i) % env.getCount()) for i in range(env.getCount())] for x in range(pow(env.getCount(), env.getCount()))]
        min_bin = env.getCount()
        min_patterns = []

        for pattern_id in range(pow(env.getCount(), env.getCount())):
            overflow = False
            env.clearBins()
            for item_id in range(env.getCount()):
                bin_id = pattern_id // pow(env.getCount(), item_id) % env.getCount()
                env.addItem(item_id, bin_id)
                if env.isOverflow():
                    overflow = True
                    break
            if not overflow:
                pattern = env.bins
                bin_c = env.countValidBin()
                if min_bin > bin_c:
                    min_patterns.clear()
                    min_bin = bin_c
                    min_patterns.append(pattern)
                    print("least pattern reached : ",pattern)
                elif min_bin == bin_c:
                    min_patterns.append(pattern)

        printPatterns(min_patterns)

Self = TypeVar("Self", bound = "ItemMask")
class ItemMask(object):
    def __init__(self, item_count : int, mask : int):
        self.mask : int = mask
        self.item_count : int = item_count


    def __or__(self, obj : Self):
        if obj.item_count != self.item_count:
            return None
        return ItemMask(self.item_count, self.mask | obj.mask)

    def __getitem__(self, key : int):
        return (self.mask >> (self.item_count - 1) - key) % 2

class BottomMask(ItemMask):
    def __init__(self, item_count : int, mask : int):
        super().__init__(item_count, mask)
        self.remain_c = item_count - self.countBottom()

    def countBottom(self):
        count = 0
        mask = self.mask
        while mask != 0:
            count += mask % 2
            mask = mask >> 1
        return count

    def isBottom(self, n):
        return self[n] == 1
    def getBinDigit(self, index):
        for i in range(self.item_count):
            if self.isBottom(i):
                if index == 0:
                    return i
                index -= 1
        return None

    

def makeDigitMask(item_count : int, digit : int):
    return ItemMask(item_count, (1 << (item_count - 1) - digit))

class BinPatterns(object):
    def __init__(self, item_count):
        self.item_count : int = item_count
        self.bottoms : int = 1
        self.pattern : int = 0
        
    def __iter__(self):
        return self

    def __next__(self):
        if self.bottoms >= 1 << self.item_count:
            raise StopIteration()
        bottom_mask = BottomMask(self.item_count, self.bottoms)
        bin_pattern = BinPattern(self.item_count, bottom_mask, self.pattern)
        
        while True:
            cont = False     
            bin_index = 0
            for i in range(self.item_count):
                if bottom_mask.isBottom(i):
                    for j in range(i + 1, self.item_count):
                        if bin_pattern[bin_index][j] == 1:
                            cont = True
                            break
                    bin_index += 1
            if not cont:
                self.incrementPattern()
                return bin_pattern
            self.incrementPattern()
            if self.bottoms >= 1 << self.item_count:
                raise StopIteration()
            bottom_mask = BottomMask(self.item_count, self.bottoms)
            bin_pattern = BinPattern(self.item_count, bottom_mask, self.pattern)

    def incrementPattern(self):
        bottom_mask = BottomMask(self.item_count, self.bottoms)
        self.pattern += 1
        if self.pattern >= pow(bottom_mask.countBottom(), bottom_mask.remain_c):
            self.bottoms += 1
            self.pattern = 0

        
class BinPattern(object):
    def __init__(self, item_count : int, bottom_mask : BottomMask, pattern : int):
        self.item_count : int = item_count
        self.bins : int = 0

        remain_c : int = bottom_mask.remain_c
        remains : list[int] = []
        for i in range (item_count):
            if not bottom_mask.isBottom(i):
                remains.append(i)

        for bin_index in range(bottom_mask.countBottom()):
            digit = bottom_mask.getBinDigit(bin_index)
            if digit is None:
                print('bottom_mask:', bottom_mask.mask, 'bin_index:', bin_index)
            if bottom_mask.isBottom(digit):
                chunk_shift = bin_index * item_count
                digit_shift = (item_count - 1) - digit
                self.bins |= 1 << chunk_shift << digit_shift
                for i in range(remain_c):
                    if pattern // pow(bottom_mask.countBottom(), i) % bottom_mask.countBottom() == bin_index:
                        digit_shift = (item_count - 1) - remains[i]
                        self.bins |= 1 << chunk_shift << digit_shift  

    def __getitem__(self, key):
        #print('bins:', self.bins, ' item_count:', self.item_count, ' key:', key)
        chunk_size = ()
        mask = (self.bins >> (key * self.item_count)) % (1 << self.item_count)
        return ItemMask(self.item_count, mask)
    


class BruteForceNoDupe:
     def solve(self, env:BinPackingEnv):
        #patterns = [[(x // pow(env.getCount(), i) % env.getCount()) for i in range(env.getCount())] for x in range(pow(env.getCount(), env.getCount()))]
        min_bin = env.getCount()
        min_patterns = []

        for pattern in BinPatterns(env.getCount()):
            overflow = False
            env.clearBins()
            counter = 0
            for bin_id in range(env.getCount()):
                for item_id in range(env.getCount()):
                    if pattern[bin_id][item_id] == 1:
                        counter += 1
                        env.addItem(item_id, bin_id)
                if env.isOverflow():
                    overflow = True
                    break
            if not overflow:
                if counter <= 6:
                    print('critical error')
                    print('checking....')
                    print('count', env.getCount())
                    for bin_id in range(env.getCount()):
                        for item_id in range(env.getCount()):
                            if pattern[bin_id][item_id] == 1:
                                print('o', end = '')
                            else:
                                print('-', end = '')
                        print()

                
                pattern = env.bins
                bin_c = env.countValidBin()
                if min_bin > bin_c:
                    min_patterns.clear()
                    min_bin = bin_c
                    min_patterns.append(pattern)
                    print("least pattern reached : ", pattern)
                elif min_bin == bin_c:
                    min_patterns.append(pattern)

        printPatterns(min_patterns)


#pattern = BinPattern(3, BottomMask(3, 3), 0)
#print(pattern.bins)

k = 0
n = 7
for pattern in BinPatterns(n):
    print('----------')
    for i in range(n):
        for j in range(n):
            if pattern[i][j] == 1:
                print('o', end='')
            else:
                print('-', end='')
        print()

#print('total loop:', k)



        

#class OptimizedBruteForceSolver:
#    def solve(self, env:BinPackingEnv):
#        for i in range(env.getCount() + 1):
#            for pattern in itertools.combinations([i for i in range(env.getCount())], ):

    

----------
ooooooo
-------
-------
-------
-------
-------
-------
----------
oooooo-
------o
-------
-------
-------
-------
-------
----------
-ooooo-
o-----o
-------
-------
-------
-------
-------
----------
o-oooo-
-o----o
-------
-------
-------
-------
-------
----------
--oooo-
oo----o
-------
-------
-------
-------
-------
----------
oo-ooo-
--o---o
-------
-------
-------
-------
-------
----------
-o-ooo-
o-o---o
-------
-------
-------
-------
-------
----------
o--ooo-
-oo---o
-------
-------
-------
-------
-------
----------
---ooo-
ooo---o
-------
-------
-------
-------
-------
----------
ooo-oo-
---o--o
-------
-------
-------
-------
-------
----------
-oo-oo-
o--o--o
-------
-------
-------
-------
-------
----------
o-o-oo-
-o-o--o
-------
-------
-------
-------
-------
----------
--o-oo-
oo-o--o
-------
-------
-------
-------
-------
----------
oo--oo-
--oo--o
-------
-------
-------
-------
-------
----------
-o--oo-
o-oo--o
-------
-------
-------
-------
---

In [158]:
class FirstFitSolver:
    def solve(self, env : BinPackingEnv):
        for item_id in range(env.getCount()):
            item_size = env.sizes[item_id]
            for bin_id in range(env.getCount()):
                bin_space = 1 - env.bin_w[bin_id]
                if bin_space >= item_size:
                    env.addItem(item_id, bin_id)
                    break
        print(env.bins)

In [159]:
class BestFitSolver:
    def solve(self, env : BinPackingEnv):
        for item_id in range(env.getCount()):
            item_size = env.sizes[item_id]
            best_bin_id = -1
            best_bin_space = 1.0
            for bin_id in range(env.getCount()):
                bin_space = 1 - env.bin_w[bin_id] - item_size
                if bin_space >= 0 and bin_space < best_bin_space:
                    best_bin_id = bin_id
                    best_bin_space = bin_space
            print(item_id, best_bin_id)
            env.addItem(item_id, best_bin_id)
        print(env.bins)

In [160]:
def greatestInRemain(sizes, space):
    greatest_size = 0
    greatest_id = -1
    for item_id in range(len(sizes)):
        item_size = sizes[item_id]
        if item_size <= space and greatest_size < item_size:
            greatest_id = item_id
            greatest_size = item_size
    return greatest_id



class KyanioSolver:
    def solve(self, env : BinPackingEnv):
        items = env.sizes.copy()
        item_ids = [i for i in range(len(items))]
        bin_id = 0
        while len(items) > 0:
            space = 1 - env.bin_w[bin_id]
            greatest_id = greatestInRemain(items, space)
            items.pop(greatest_id)
            item_id = item_ids.pop(greatest_id)
            env.addItem(item_id, bin_id)
            while True:
                space = 1 - env.bin_w[bin_id]
                greatest_id = greatestInRemain(items, space)
                if greatest_id == -1:
                    break
                items.pop(greatest_id)
                item_id = item_ids.pop(greatest_id)
                env.addItem(item_id, bin_id)
            bin_id += 1
        
        print(env.bins)




Test

In [161]:
%%time

#問題を解くためのシミュレート環境を作成
bpenv = BinPackingEnv(sizes)


c = bpenv.countValidBin()
#アルゴリズム1総当たり
#solver1 = BruteForceSolverSingleLoop()
#実行
#solver1.solve(bpenv)

solver_nd = BruteForceNoDupe()
solver_nd.solve(bpenv)

#solver2 = FirstFitSolver()
#solver2.solve(bpenv)

#solver3 = BestFitSolver()
#solver3.solve(bpenv)

#solver4 = KyanioSolver()
#solver4.solve(bpenv)

least pattern reached :  [[2, 3, 4], [0, 1, 5], [6], [], [], [], []]
----------
[2, 3, 4]
[0, 1, 5]
[6]
[]
[]
[]
[]
----------
[2, 3, 4]
[1, 5]
[0, 6]
[]
[]
[]
[]
----------
[2, 3, 4]
[0, 5]
[1, 6]
[]
[]
[]
[]
----------
[1, 3, 4]
[0, 2, 5]
[6]
[]
[]
[]
[]
----------
[1, 3, 4]
[2, 5]
[0, 6]
[]
[]
[]
[]
----------
[0, 3, 4]
[2, 5]
[1, 6]
[]
[]
[]
[]
----------
[3, 4]
[0, 2, 5]
[1, 6]
[]
[]
[]
[]
----------
[1, 3, 4]
[0, 5]
[2, 6]
[]
[]
[]
[]
----------
[0, 3, 4]
[1, 5]
[2, 6]
[]
[]
[]
[]
----------
[3, 4]
[0, 1, 5]
[2, 6]
[]
[]
[]
[]
----------
[1, 2, 4]
[0, 3, 5]
[6]
[]
[]
[]
[]
----------
[1, 2, 4]
[3, 5]
[0, 6]
[]
[]
[]
[]
----------
[0, 2, 4]
[3, 5]
[1, 6]
[]
[]
[]
[]
----------
[2, 4]
[0, 3, 5]
[1, 6]
[]
[]
[]
[]
----------
[0, 1, 4]
[3, 5]
[2, 6]
[]
[]
[]
[]
----------
[1, 4]
[0, 3, 5]
[2, 6]
[]
[]
[]
[]
----------
[1, 2, 4]
[0, 5]
[3, 6]
[]
[]
[]
[]
----------
[0, 2, 4]
[1, 5]
[3, 6]
[]
[]
[]
[]
----------
[2, 4]
[0, 1, 5]
[3, 6]
[]
[]
[]
[]
----------
[0, 1, 4]
[2, 5]
[3, 6]
[]
